## 作業目標
* 嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [1]:
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [4]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

#### SVM model
* SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [該知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

#### 用 histogram 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [5]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

#### 用 HOG 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [6]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

In [7]:
from sklearn.metrics import accuracy_score

score_train_hist = accuracy_score(y_true=y_train, y_pred=y_hist_train)
score_test_hist = accuracy_score(y_true=y_test, y_pred=y_hist_test)

score_train_hog = accuracy_score(y_true=y_train, y_pred=y_hog_train)
score_test_hog = accuracy_score(y_true=y_test, y_pred=y_hog_test)

print('Histogram score-')
print('Train set: ', score_train_hist)
print('Test set : ', score_test_hist)

print('Hog score-')
print('Train set: ', score_train_hog)
print('Test set : ', score_test_hog)

Histogram score-
Train set:  0.14846
Test set :  0.1448
Hog score-
Train set:  0.20592
Test set :  0.2058


In [10]:
from sklearn.metrics import confusion_matrix

conmat_test_hist = confusion_matrix(y_true=y_test, y_pred=y_hist_test)
print('Confusion Matrix of Histogram')
print(conmat_test_hist)
print()
print('Confusion Matrix of Hog')
conmat_test_hog = confusion_matrix(y_true=y_test, y_pred=y_hog_test)
print(conmat_test_hog)

Confusion Matrix of Histogram
[[383  67 255  60  43  58   6  20  99   9]
 [174  50 221  61  67 248   3   7 135  34]
 [189  35 232 116 225 118  16  15  41  13]
 [198  25 199 103 130 221   7  10  79  28]
 [ 97  15 237 130 327 127  10  10  40   7]
 [203  16 230 120 117 194   8   9  92  11]
 [ 95  24 210 113 292 190  18  25  26   7]
 [165  30 284 107 167 148   3  12  66  18]
 [382  18 281  72  34  76   0   3 119  15]
 [213  50 345  55  47 153   1   2 124  10]]

Confusion Matrix of Hog
[[ 79 267 153  38  87  88  10  95  42 141]
 [ 31 489  17  43  32  25  11  89  35 228]
 [ 30  79 255 194  77 149   3 129  15  69]
 [ 47  86  94 314  51 165  20 144   6  73]
 [ 43  67 180 190  78 202  10 145   5  80]
 [ 18  67 144 321  40 181  12 173   1  43]
 [ 32 104 113 207  28 195  14 206  18  83]
 [ 16 111  69 203  47 173  14 280   6  81]
 [ 21 388  90  17  85  42   2  88  45 222]
 [ 28 290  36  67  22  42   3 170  19 323]]
